In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None

# COVID

In [2]:
OCSE_ISO = [
    "AUS",
    "AUT",
    "BEL",
    "BRA",
    "CAN",
    "CHL",
    "CHN",
    "CZE",
    "COL",
    "CRI",
    "DNK",
    "EST",
    "FIN",
    "FRA",
    "DEU",
    "GRC",
    "HUN",
    "ISL",
    "IND",
    "IDN",
    "IRL",
    "ISR",
    "ITA",
    "KOR",
    "JPN",
    "LVA",
    "LTU",
    "LUX",
    "MEX",
    "NLD",
    "NZL",
    "NOR",
    "POL",
    "PRT",
    "SVK",
    "SVN",
    "ZAF",
    "ESP",
    "SWE",
    "CHE",
    "TUR",
    "GBR",
    "USA",
]

wave1 = {
    "name": "Wave 1",
    "start": "2020-02-01",
    "end": "2020-06-30",
}

wave2 = {
    "name": "Wave 2",
    "start": "2020-09-01",
    "end": "2020-12-31",
}

wave3 = {
    "name": "Wave 3",
    "start": "2021-02-01",
    "end": "2021-06-30",
}

wave4 = {
    "name": "Wave 4",
    "start": "2021-09-01",
    "end": "2021-12-31",
}

In [3]:
covid_input_data = pd.read_csv("dataset/original/our_world_in_data_input.csv", usecols=[
    "continent",
    "location",
    "iso_code",
    "date",
    "new_cases",
    "new_deaths",
    "icu_patients",
    "new_tests",
    "new_vaccinations",
    "population_density",
    "population",
    "gdp_per_capita",
    "median_age",
    "gdp_per_capita",
])

covid_input_data.head()

,iso_code,continent,location,date,new_cases,new_deaths,icu_patients,new_tests,new_vaccinations,population,population_density,median_age,gdp_per_capita
0,AFG,Asia,Afghanistan,2020-02-24,5.0,NaN,NaN,NaN,NaN,39835428.0,54.422,18.6,1803.987
1,AFG,Asia,Afghanistan,2020-02-25,0.0,NaN,NaN,NaN,NaN,39835428.0,54.422,18.6,1803.987
2,AFG,Asia,Afghanistan,2020-02-26,0.0,NaN,NaN,NaN,NaN,39835428.0,54.422,18.6,1803.987
3,AFG,Asia,Afghanistan,2020-02-27,0.0,NaN,NaN,NaN,NaN,39835428.0,54.422,18.6,1803.987
4,AFG,Asia,Afghanistan,2020-02-28,0.0,NaN,NaN,NaN,NaN,39835428.0,54.422,18.6,1803.987


In [4]:
mask = covid_input_data["iso_code"].apply(lambda x: any(item for item in [*OCSE_ISO] if item in x))

covid_ocse_data = covid_input_data[mask]

covid_ocse_data.head()

,iso_code,continent,location,date,new_cases,new_deaths,icu_patients,new_tests,new_vaccinations,population,population_density,median_age,gdp_per_capita
7209,AUS,Oceania,Australia,2020-01-26,4.0,NaN,NaN,NaN,NaN,25788217.0,3.202,37.9,44648.71
7210,AUS,Oceania,Australia,2020-01-27,1.0,NaN,NaN,NaN,NaN,25788217.0,3.202,37.9,44648.71
7211,AUS,Oceania,Australia,2020-01-28,0.0,NaN,NaN,NaN,NaN,25788217.0,3.202,37.9,44648.71
7212,AUS,Oceania,Australia,2020-01-29,1.0,NaN,NaN,NaN,NaN,25788217.0,3.202,37.9,44648.71
7213,AUS,Oceania,Australia,2020-01-30,3.0,NaN,NaN,NaN,NaN,25788217.0,3.202,37.9,44648.71


In [5]:
def sum_columns(data, columns_to_be_summed, waves):    
    rows = []

    for wave_number, wave in enumerate(waves):    
        dates = pd.to_datetime(data['date'])

        mask = (dates >= wave["start"]) & (dates <= wave["end"])

        filtered_data = data[mask]

        first = True
        previous_location = ""
        
        for row_number, row in data.iterrows():

            if first:
                starting_row = row_number
                first = False

            location = row["location"]

            if row_number != starting_row and location == previous_location:
                continue
                        
            rows.append(
                tuple(["{}".format(wave_number + 1)]) +
                tuple([row[row_column] for row_column in data if row_column not in columns_to_be_summed]) +
                tuple([filtered_data.loc[filtered_data["location"] == row["location"]][row_column].sum() for row_column in columns_to_be_summed])
            )

            previous_location = location
    
    output_columns = ["wave"] + [column for column in data if column not in columns_to_be_summed] + columns_to_be_summed

    return pd.DataFrame(rows, columns=output_columns).drop(columns=["date"])

In [6]:
output_data = sum_columns(covid_ocse_data, ["new_cases", "new_deaths", "new_tests", "icu_patients", "new_vaccinations"], [wave1, wave2, wave3, wave4])

output_data.to_csv("dataset/generated/covid_data.csv", index=False)

output_data.head()

print("Dataset generated in dataset/generated/covid_data.csv!")

Dataset generated in dataset/generated/covid_data.csv!


# Regimes

In [7]:
political_regimes_input_data = pd.read_csv("dataset/original/political_regimes.csv", usecols=[
    "Entity", 
    "Year", 
    "Code",
    "Political regime", 
])

political_regimes_input_data.head()

mask = political_regimes_input_data["Code"].apply(lambda x: any(item for item in [*OCSE_ISO] if item in x))

political_regimes_ocse_data = political_regimes_input_data[mask]

ocse_data2019 = political_regimes_ocse_data.loc[political_regimes_ocse_data["Year"] == 2019]

ocse_data2019.rename(columns={
    "Entity": "location",
    "Code": "iso_code",
    "Year": "year",
    "Political regime": "political_regime",
}, inplace=True)

ocse_data2019.to_csv("dataset/generated/political_regime_data.csv", index=False, columns=["location", "iso_code", "political_regime"])

ocse_data2019.head()

print("Dataset generated in dataset/generated/covid_data.csv!")

Dataset generated in dataset/generated/covid_data.csv!


# Corruption

In [8]:
cpi_data_2019 = pd.read_excel("dataset/original/cpi2019.xlsx", usecols=[
    "Country",
    "CPI",
    "ISO3",
])

mask = cpi_data_2019["ISO3"].apply(lambda x: any(item for item in [*OCSE_ISO] if item in x))

ocse_cpi_data = cpi_data_2019[mask]

ocse_cpi_data.rename(columns={
    "Country": "location",
    "ISO3": "iso_code",
    "CPI": "cpi",
}, inplace=True)

ocse_cpi_data.to_csv("dataset/generated/corruption_data.csv", index=False)

ocse_cpi_data.head()

print("Dataset generated in dataset/generated/corruption_data.csv!")


Dataset generated in dataset/generated/corruption_data.csv!


In [9]:
corruption_control_input = pd.read_csv("dataset/original/corruption_control2019.csv", usecols=[
    "Country Name",
    "Country Code",
    "2019 [YR2019]"
])

mask = corruption_control_input["Country Code"].apply(lambda x: any(item for item in [*OCSE_ISO] if item in x))

ocse_corruption_control_data = corruption_control_input[mask]

ocse_corruption_control_data.rename(columns={
    "Country Name": "location",
    "Country Code": "iso_code",
    "2019 [YR2019]": "corruption_control",
}, inplace=True)

ocse_corruption_control_data.head()

ocse_corruption_control_data.to_csv("dataset/generated/corruption_control.csv", index=False)

# Healthcare expenditure

In [10]:
healtchare_expenditure_input_data = pd.read_csv("dataset/original/healthcare_expenditure.csv", usecols=[
    "LOCATION", 
    "SUBJECT",
    "MEASURE",
    "TIME",
    "Value"
])

mask = healtchare_expenditure_input_data["LOCATION"].apply(lambda x: any(item for item in [*OCSE_ISO] if item in x))

ocse_healtchare_expenditure = healtchare_expenditure_input_data[mask]

ocse_healtchare_expenditure2019 = ocse_healtchare_expenditure.loc[
    (ocse_healtchare_expenditure["TIME"] == 2018) 
    & (ocse_healtchare_expenditure["SUBJECT"] == "TOT")
    & (ocse_healtchare_expenditure["MEASURE"] == "USD_CAP")
]

ocse_healtchare_expenditure2019.rename(columns={
    "LOCATION": "iso_code",
    "TIME": "year",
    "Value": "healthcare_expenditure",
}, inplace=True)

ocse_healtchare_expenditure2019.to_csv("dataset/generated/ocse_healthcare_expenditure.csv", index=False, columns=["iso_code", "healthcare_expenditure"])

ocse_healtchare_expenditure2019.head()

print("Dataset generated in dataset/generated/ocse_healthcare_expenditure.csv!")

Dataset generated in dataset/generated/ocse_healthcare_expenditure.csv!


# Arrivals

In [11]:
number_of_arrivals_input_data = pd.read_csv("dataset/original/number_arrivals.csv",  usecols=[
    "Country Name",
    "Country Code", 
    "2019",
])

mask = number_of_arrivals_input_data["Country Code"].apply(lambda x: any(item for item in [*OCSE_ISO] if item in x))

ocse_number_of_arrivals2019 = number_of_arrivals_input_data[mask]

ocse_number_of_arrivals2019.rename(columns={
    "Country Name": "location",
    "Country Code": "iso_code",
    "2019": "number_of_arrivals",
}, inplace=True)

ocse_number_of_arrivals2019.to_csv("dataset/generated/ocse_number_arrivals.csv", index=False)

ocse_number_of_arrivals2019.head()

,location,iso_code,number_of_arrivals
13,Australia,AUS,9466000.0
14,Austria,AUT,31884000.0
17,Belgium,BEL,9343000.0
29,Brazil,BRA,6353000.0
35,Canada,CAN,32430000.0


# Governament effectiveness

In [12]:
governament_effectiveness_input = pd.read_csv("dataset/original/governance_effectiveness.csv", usecols=[
    "Country Name",
    "Country Code",
    "2019 [YR2019]"
])

mask = governament_effectiveness_input["Country Code"].apply(lambda x: any(item for item in [*OCSE_ISO] if item in x))

ocse_governament_effectiveness_data = governament_effectiveness_input[mask]

ocse_governament_effectiveness_data.rename(columns={
    "Country Name": "location",
    "Country Code": "iso_code",
    "2019 [YR2019]": "governament_effectiveness",
}, inplace=True)

ocse_governament_effectiveness_data.to_csv("dataset/generated/governance_effectiveness.csv", index=False)

ocse_governament_effectiveness_data.head()

,location,iso_code,governament_effectiveness
11,Australia,AUS,1.571555
12,Austria,AUT,1.527737
19,Belgium,BEL,1.148132
27,Brazil,BRA,-0.1869027
35,Canada,CAN,1.731293


# Trust in governance

In [13]:
trust_in_governance_input = pd.read_csv("dataset/original/trust_in_governance.csv", usecols=[
    "LOCATION",
    "TIME",
    "Value",
])

mask = trust_in_governance_input["LOCATION"].apply(lambda x: any(item for item in [*OCSE_ISO] if item in x))

trust_in_goverance_ocse2019 = trust_in_governance_input[mask].loc[trust_in_governance_input["TIME"] == 2019]

trust_in_goverance_ocse2019.rename(columns={
    "LOCATION": "iso_code",
    "TIME": "year",
    "Value": "trust_in_governance",
}, inplace=True)

trust_in_goverance_ocse2019.to_csv("dataset/generated/trust_in_governance.csv", index=False, columns=["iso_code", "trust_in_governance"])

trust_in_goverance_ocse2019.head()

,iso_code,year,trust_in_governance
12,AUS,2019,46.870001
26,AUT,2019,51.233826
40,BEL,2019,32.790115
55,CAN,2019,54.928097
83,DNK,2019,63.299903


# Merge

In [18]:
covid_data = pd.read_csv("dataset/generated/covid_data.csv")

covid_data.drop(columns=["continent"], inplace=True)

corruption_control_data = pd.read_csv("dataset/generated/corruption_control.csv")
cpi_data = pd.read_csv("dataset/generated/corruption_data.csv")
governament_effectiveness_data = pd.read_csv("dataset/generated/governance_effectiveness.csv")
healthcare_expenditure_data = pd.read_csv("dataset/generated/ocse_healthcare_expenditure.csv")
number_of_arrivals_data = pd.read_csv("dataset/generated/ocse_number_arrivals.csv")
political_regime_data = pd.read_csv("dataset/generated/political_regime_data.csv")
trust_in_governance_data = pd.read_csv("dataset/generated/trust_in_governance.csv")
government_type_data = pd.read_csv("dataset/generated/government_type.csv")

government_type_data.head()

corruption_merge = pd.merge(corruption_control_data, cpi_data, on=["iso_code", "location"], how="inner")
governament_effectiveness_merge = pd.merge(corruption_merge, governament_effectiveness_data, on=["iso_code", "location"], how="inner")
healthcare_merge = pd.merge(governament_effectiveness_merge, healthcare_expenditure_data, on=["iso_code"], how="inner")
number_of_arrival_merge = pd.merge(healthcare_merge, number_of_arrivals_data, on=["iso_code", "location"], how="inner")
political_regime_merge = pd.merge(number_of_arrival_merge, political_regime_data, on=["iso_code", "location"], how="inner")
political_data = pd.merge(political_regime_merge, trust_in_governance_data, on=["iso_code"], how="inner")
government_type_data = pd.merge(political_data, government_type_data, on=["iso_code", "location"], how="inner")

covid_data_wave1 = covid_data.loc[covid_data["wave"] == "wave_1"]

output_data = pd.merge(government_type_data, covid_data, on=["iso_code", "location"], how="inner")

output_data.to_csv("dataset/dataset.csv", index=False)

output_data.head()

,location,iso_code,corruption_control,cpi,governament_effectiveness,healthcare_expenditure,number_of_arrivals,political_regime,trust_in_governance,government_type,wave,population,population_density,median_age,gdp_per_capita,new_cases,new_deaths,new_tests,icu_patients,new_vaccinations
0,Australia,AUS,1.825413,77,1.571555,4793.460,9466000.0,3,46.870001,1,1,25788217.0,3.202,37.9,44648.710,7911.0,104.0,2158828.0,0.0,0.0
1,Australia,AUS,1.825413,77,1.571555,4793.460,9466000.0,3,46.870001,1,2,25788217.0,3.202,37.9,44648.710,2606.0,252.0,2641803.0,0.0,0.0
2,Australia,AUS,1.825413,77,1.571555,4793.460,9466000.0,3,46.870001,1,3,25788217.0,3.202,37.9,44648.710,1825.0,1.0,3768641.0,0.0,7298477.0
3,Australia,AUS,1.825413,77,1.571555,4793.460,9466000.0,3,46.870001,1,4,25788217.0,3.202,37.9,44648.710,180454.0,1105.0,11712836.0,0.0,20829569.0
4,Austria,AUT,1.557738,77,1.527737,5544.767,31884000.0,3,51.233826,2,1,9043072.0,106.749,44.4,45436.686,17766.0,705.0,595922.0,8468.0,0.0
